In [2]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Función para aceptar cookies ---
def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(1)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")

# --- Función para extraer coordenadas ---
def extraer_coords(src):
    match = re.search(r"/(\d+_\d+_)([0-9.]+)@(-?[0-9.]+)_", src)
    if match:
        lat, lon = match.group(2), match.group(3)
        return lat, lon
    return None, None

# --- Leer archivo CSV con los enlaces ---
df = pd.read_csv(r"C:\Users\pablo\proyecto\comparador\venta_madrid_detalle_completo_limpio.csv")
urls = df["link"].dropna().unique().tolist()[:5]  # <- SOLO 5 PARA PRUEBA

# --- Configuración de Selenium ---
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Puedes quitarlo si quieres ver la ventana
service = Service()
driver = webdriver.Chrome(service=service, options=options)

print(f"🔍 Scrapeando coordenadas para {len(urls)} enlaces...\n")
datos = []

for i, url in enumerate(urls, 1):
    print(f"{i}/{len(urls)} → {url}")
    driver.get(url)
    time.sleep(2)

    if i == 1:
        aceptar_cookies(driver)

    try:
        imagen = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img[alt='mapa']"))
        )
        src = imagen.get_attribute("src")
        print("🧭 src:", src)
        lat, lon = extraer_coords(src)
    except:
        print("❌ No se encontró el mapa.")
        lat, lon = None, None

    datos.append({"link": url, "lat": lat, "lon": lon})

driver.quit()

# --- Guardar o mostrar resultados ---
df_coords = pd.DataFrame(datos)
df_coords.to_csv("coordenadas_venta_scrapeadas.csv", index=False)
print("\n✅ Coordenadas guardadas en coordenadas_venta_scrapeadas.csv")
df_coords.head()


🔍 Scrapeando coordenadas para 5 enlaces...

1/5 → https://www.pisos.com/comprar/casa_adosada-estacion_zona_norte28224-5106780590_109700/
⚠️ Cookies ya aceptadas o no detectadas.
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.4492938@-3.8070858_1_0.gif
2/5 → https://www.pisos.com/comprar/piso-san_sebastian_de_los_reyes_casco_antiguo28703-53387680450_515177/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.545353@-3.6276307_1_0.gif
3/5 → https://www.pisos.com/comprar/piso-zona_sureste-5129162041_109700/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.4555498@-3.4545642_1_0.gif
4/5 → https://www.pisos.com/comprar/piso-hoyo_de_manzanares_centro_urbano-46711883004_994129/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.622730436115255@-3.9071235135966056_1_0.gif
5/5 → https://www.pisos.com/comprar/piso-sol_barrio28013-53433152926_524671/
🧭 src: https://map.imghs.net/Cache/Z2R35MG0MC1/2_350_40.4174162@-3.7038621_0_1.gif

✅ Coordenadas guardadas en coordenadas_

,link,lat,lon
0,https://www.pisos.com/comprar/casa_adosada-est...,40.4492938,-3.8070858
1,https://www.pisos.com/comprar/piso-san_sebasti...,40.545353,-3.6276307
2,https://www.pisos.com/comprar/piso-zona_surest...,40.4555498,-3.4545642
3,https://www.pisos.com/comprar/piso-hoyo_de_man...,40.622730436115255,-3.9071235135966056
4,https://www.pisos.com/comprar/piso-sol_barrio2...,40.4174162,-3.7038621


In [6]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Aceptar cookies si aparecen ---
def aceptar_cookies(driver):
    try:
        wait = WebDriverWait(driver, 10)
        boton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#didomi-notice-agree-button")))
        boton.click()
        print("🍪 Cookies aceptadas.")
        time.sleep(1)
    except:
        print("⚠️ Cookies ya aceptadas o no detectadas.")

# --- Extraer coordenadas del src del mapa ---
def extraer_coords(src):
    match = re.search(r"/(\d+_\d+_)([0-9.]+)@(-?[0-9.]+)_", src)
    if match:
        lat, lon = match.group(2), match.group(3)
        return lat, lon
    return None, None

# --- Leer CSV completo de venta ---
df = pd.read_csv(r"C:\Users\pablo\proyecto\comparador\venta_madrid_detalle_completo_limpio.csv")
urls = df["link"].dropna().unique().tolist()  # 🔓 Ya SIN límite

# --- Configuración Selenium ---
options = webdriver.ChromeOptions()
options.add_argument("--headless")
service = Service()
driver = webdriver.Chrome(service=service, options=options)

datos = []

print(f"🔍 Scrapeando coordenadas para {len(urls)} enlaces...\n")

for i, url in enumerate(urls, 1):
    print(f"{i}/{len(urls)} → {url}")
    driver.get(url)
    time.sleep(2)

    if i == 1:
        aceptar_cookies(driver)

    try:
        imagen = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "img[alt='mapa']"))
        )
        src = imagen.get_attribute("src")
        print("🧭 src:", src)
        lat, lon = extraer_coords(src)
    except:
        print("❌ No se encontró el mapa.")
        lat, lon = None, None

    datos.append({"link": url, "lat": lat, "lon": lon})

driver.quit()

# --- Guardar resultados ---
df_coords = pd.DataFrame(datos)
df_coords.to_csv("coordenadas_venta_scrapeadas.csv", index=False)
print("\n✅ Coordenadas guardadas en coordenadas_venta_scrapeadas.csv")
df_coords.head()


🔍 Scrapeando coordenadas para 3430 enlaces...

1/3430 → https://www.pisos.com/comprar/casa_adosada-estacion_zona_norte28224-5106780590_109700/
⚠️ Cookies ya aceptadas o no detectadas.
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.4492938@-3.8070858_1_0.gif
2/3430 → https://www.pisos.com/comprar/piso-san_sebastian_de_los_reyes_casco_antiguo28703-53387680450_515177/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.545353@-3.6276307_1_0.gif
3/3430 → https://www.pisos.com/comprar/piso-zona_sureste-5129162041_109700/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.4555498@-3.4545642_1_0.gif
4/3430 → https://www.pisos.com/comprar/piso-hoyo_de_manzanares_centro_urbano-46711883004_994129/
🧭 src: https://map.imghs.net/Cache/Z1R35MG1MC0/1_350_40.622730436115255@-3.9071235135966056_1_0.gif
5/3430 → https://www.pisos.com/comprar/piso-sol_barrio28013-53433152926_524671/
🧭 src: https://map.imghs.net/Cache/Z2R35MG0MC1/2_350_40.4174162@-3.7038621_0_1.gif
6/3430 → https://www.p

,link,lat,lon
0,https://www.pisos.com/comprar/casa_adosada-est...,40.4492938,-3.8070858
1,https://www.pisos.com/comprar/piso-san_sebasti...,40.545353,-3.6276307
2,https://www.pisos.com/comprar/piso-zona_surest...,40.4555498,-3.4545642
3,https://www.pisos.com/comprar/piso-hoyo_de_man...,40.622730436115255,-3.9071235135966056
4,https://www.pisos.com/comprar/piso-sol_barrio2...,40.4174162,-3.7038621


In [18]:
import pandas as pd

# 📥 1. Cargar los archivos CSV
df_json = pd.read_csv("C:/Users/pablo/proyecto/comparador/venta_madrid_detalle_completo_limpio.csv")
df_coords = pd.read_csv("C:/Users/pablo/proyecto/coordenadas_venta_scrapeadas.csv")

# 🔗 2. Unir los dataframes usando la columna 'link'
df_merged = pd.merge(df_json, df_coords, on="link", how="left")

# ✅ 3. Mostrar los primeros resultados
print(df_merged.head())

# 💾 4. Guardar el resultado si lo necesitas
df_merged.to_csv("C:/Users/pablo/proyecto/comparador/venta_madrid_con_coordenadas.csv", index=False)



                    timestamp  \
0  2025-04-16 11:20:56.879225   
1  2025-04-16 11:20:56.949513   
2  2025-04-16 11:20:57.009922   
3  2025-04-16 11:20:57.079706   
4  2025-04-16 11:20:57.149550   

                                              titulo  \
0  Casa adosada en calle de los Hermanos San Romá...   
1                              Piso en calle Real, 9   
2                     Piso en calle de la Plata, s/n   
3      Piso en calle Urbanización El Roquedal, nº 13   
4                            Piso en calle de Tetuán   

                                    ubicacion  \
0    Estación-Zona Norte (Pozuelo de Alarcón)   
1  Casco Antiguo (San Sebastián de los Reyes)   
2            Zona Sureste (Torrejón de Ardoz)   
3                          Hoyo de Manzanares   
4       Sol (Distrito Centro. Madrid Capital)   

                                                link  \
0  https://www.pisos.com/comprar/casa_adosada-est...   
1  https://www.pisos.com/comprar/piso-san_sebasti...   
2

In [29]:
df_venta = pd.read_csv("C:/Users/pablo/proyecto/alquiler/alquiler_madrid_completo_final.csv")
df_venta.head()


,timestamp_x,titulo_x,precio_x,ubicacion_x,habitaciones_x,baños_x,metros_x,link,timestamp_y,titulo_y,precio_y,ubicacion_y,habitaciones_y,baños_y,metros_y,descripcion,superficie_construida,superficie_util,antiguedad,imagen
0,2025-04-14T11:21:19.998558,Piso en calle Luis Garcia Cereceda,8.500 €/mes,La Finca (Pozuelo de Alarcón),3 habs.,3 baños,378 m²,https://www.pisos.com/alquilar/piso-la_finca-4...,1744629679998,Piso en calle Luis Garcia Cereceda,8.500 €/mes,La Finca (Pozuelo de Alarcón),3 habs.,3 baños,378 m²,Traducciones disponibles:\n Español\n Ca...,378 m²,188 m²,Menos de 5 años,https://fotos.imghs.net/fch-wp/2805/4917840215...
1,2025-04-14T11:21:20.078450,Piso en calle Libertad,1.450 €/mes,La Navata (Galapagar),3 habs.,1 baño,90 m²,https://www.pisos.com/alquilar/piso-galapagar_...,1744629680078,Piso en calle Libertad,1.450 €/mes,La Navata (Galapagar),3 habs.,1 baño,90 m²,Traducciones disponibles:\n Español\n Ca...,90 m²,87 m²,Menos de 5 años,https://fotos.imghs.net/fch-wp/1005/639/1005_1...
2,2025-04-14T11:21:20.162385,Casa unifamiliar en calle XIX,4.000 €/mes,Las Matas-Los Peñascales (Las Rozas de Madrid),6 habs.,7 baños,600 m²,https://www.pisos.com/alquilar/casa_unifamilia...,1744629680162,Casa unifamiliar en calle XIX,4.000 €/mes,Las Matas-Los Peñascales (Las Rozas de Madrid),6 habs.,7 baños,600 m²,Traducciones disponibles:\n Español\n Ca...,600 m²,600 m²,Entre 20 y 30 años,https://fotos.imghs.net/fch-wp/2805/2918987808...
3,2025-04-14T11:21:20.243002,Piso en calle de Ponzano,3.500 €/mes,Río Rosas (Distrito Chamberí. Madrid Capital),4 habs.,3 baños,226 m²,https://www.pisos.com/alquilar/piso-rios_rosas...,1744629680243,Piso en calle de Ponzano,3.500 €/mes,Río Rosas (Distrito Chamberí. Madrid Capital),4 habs.,3 baños,226 m²,NaN,NaN,NaN,NaN,NaN
4,2025-04-14T11:21:20.312848,Piso en Zona Avenida de Europa,1.400 €/mes,Zona Avenida de Europa (Pozuelo de Alarcón),1 hab.,1 baño,65 m²,https://www.pisos.com/alquilar/piso-zona_aveni...,1744629680312,Piso en Zona Avenida de Europa,1.400 €/mes,Zona Avenida de Europa (Pozuelo de Alarcón),1 hab.,1 baño,65 m²,Traducciones disponibles:\n Español\n Ca...,65 m²,NaN,NaN,https://fotos.imghs.net/fch-wp/1018/798/1018_1...
